In [10]:
# import all modules, some might not be necessary (DIE NOG VERWIJDEREN)
from ema_workbench import (
    Model,
    MultiprocessingEvaluator,
    ScalarOutcome,
    IntegerParameter,
    optimize,
    Scenario,
    Constraint,
)
from ema_workbench.em_framework.optimization import EpsilonProgress, epsilon_nondominated
from ema_workbench.util import ema_logging

from ema_workbench import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem, ArchiveLogger
from custom_problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import pandas as pd


In [2]:
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [14]:
model, steps = get_model_for_problem_formulation()
problem = to_problem(model, searchover="levers")

In [15]:
# load the selected scenarios
scenarios_df = pd.read_csv('results/selected_scenarios.csv')
scenarios_df.rename(columns={'Unnamed: 0': 'scenario_id'}, inplace=True)
scenarios_df = scenarios_df.set_index('scenario_id')
scenarios_df

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,A.4_Bmax,A.4_Brate,A.4_pfail,A.5_Bmax,A.5_Brate,A.5_pfail,discount rate 0,discount rate 1,discount rate 2
scenario_id,,,,,,,,,,,,,,,,,,,
12275,26,210.807918,1.5,0.092027,172.374261,1.5,0.517100,254.006564,1.0,0.915031,165.440227,10.0,0.235617,183.021862,10.0,0.342190,2.5,1.5,3.5
39833,101,123.723596,10.0,0.087285,315.055206,1.0,0.019649,34.954835,10.0,0.305029,145.432329,1.0,0.862907,209.075297,1.5,0.009042,1.5,3.5,3.5
35280,119,84.948477,10.0,0.294570,305.283047,10.0,0.702650,220.064913,1.0,0.326005,49.619647,1.5,0.009365,306.976860,10.0,0.426481,4.5,3.5,1.5
20422,20,323.962775,1.5,0.190418,328.161863,10.0,0.014162,302.061696,10.0,0.051706,289.845385,1.5,0.059208,83.701692,1.0,0.692434,1.5,1.5,3.5
4717,104,40.525349,10.0,0.161001,187.789168,1.5,0.010373,205.979011,1.0,0.958783,340.402481,1.0,0.086792,75.494126,1.5,0.826855,1.5,1.5,1.5


In [39]:
scenario_ids = scenarios_df.index.to_list()
archive_list = []
results_list = []
convergence_list = []
amount_of_seeds = 4

for scenario in scenario_ids:
    for i in range(amount_of_seeds):
        archives = ArchiveLogger.load_archives(f"./archive/Policy_search_arch_{float(scenario)}_seed{i}.tar.gz")
        for key, df in archives.items():
            if 'Unnamed: 0' in df.columns:
                del df['Unnamed: 0']
        archive_list.append(archives)

        result = pd.read_csv(f"./results/Policy_search_scen{float(scenario)}_seed{i}results.csv", index_col=0)
        results_list.append(result)

        convergence = pd.read_csv(f"./results/Policy_search_scen{float(scenario)}_seed{i}convergence.csv", index_col=0)
        convergence_list.append(convergence)

archive_dict = {}
results_dict = {}
convergence_dict = {}  
for i in range(len(scenario_ids)):
    archive_dict[scenario_ids[i]] = archive_list[i]
    results_dict[scenario_ids[i]] = [results_list[i]]
    convergence_dict[scenario_ids[i]] = convergence_list[i]   

In [44]:
results_dict[39833]

[    0_RfR 0  0_RfR 1  0_RfR 2  1_RfR 0  1_RfR 1  1_RfR 2  2_RfR 0  2_RfR 1  \
 0         1        0        0        1        0        1        0        1   
 1         0        1        1        1        0        1        1        1   
 2         1        0        1        0        0        0        1        1   
 3         0        0        1        1        0        0        1        1   
 4         1        0        0        1        0        0        1        1   
 5         1        0        1        0        0        0        1        1   
 6         1        0        0        0        0        1        1        1   
 7         1        0        1        0        0        0        1        1   
 8         1        0        1        0        0        0        1        1   
 9         1        0        1        0        0        0        1        1   
 10        1        0        1        0        0        0        1        1   
 11        1        0        1        0        0    

In [45]:
policy_dict = {}
epsilon = [100, 0.01, 100, 100, 0.01] # [[0.05] * len(model.outcomes)]   # [100, 0.01, 100, 100, 0.01] # OF, [0.05] * len(model.outcomes)
for i in (scenario_ids):
    print(i)
    df = epsilon_nondominated(results_dict[i], epsilon, problem)
    policy_dict[scenario_ids[i]] = df
    n_policies = df.shape[0]
    print(f"Scenario {scenario_ids[i]} has {n_policies} non-dominated policies")

12275


EMAError: parameter names ['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2'] not found in archive

In [38]:
policy_sets = {}
epsilon = [100, 0.01, 100, 100, 0.01]
for scenario in scenario_ids:
    df = epsilon_nondominated(results_dict[scenario], epsilon, problem)
    policy_sets[scenario] = df
    n_policies = df.shape[0]
    print(f"Scenario {scenario} has {n_policies} non-dominated policies")

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [34]:
# Define the objective columns
objective_columns = [
    "A.1_External Costs",
    "A.1_Expected Number of Deaths",
    "A.1_Expected Annual Damage",
    "A.2_External Costs",
    "A.2_Expected Number of Deaths",
    "A.2_Expected Annual Damage"
]

# Check if solution1 dominates solution2
def dominates(solution1, solution2):
    return all(x <= y for x, y in zip(solution1, solution2)) and any(x < y for x, y in zip(solution1, solution2))

# Find non-dominated solutions
non_dominated = []

for i in range(len(df)):
    is_dominated = False
    for j in range(len(df)):
        if i != j and dominates(df.loc[j, objective_columns], df.loc[i, objective_columns]):
            is_dominated = True
            break
    if not is_dominated:
        non_dominated.append(i)

# Extract non-dominated solutions
non_dominated_df = df.iloc[non_dominated]

# Display non-dominated solutions
print(non_dominated_df)

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.1_External Costs,A.1_RfR Costs,A.1_Expected Number of Deaths,A.1_Expected Annual Damage,A.2_External Costs,A.2_RfR Costs,A.2_Expected Number of Deaths,A.2_Expected Annual Damage
0,1,1,1,1,1,0,1,1,1,1,...,2,8,0.0,0,0.000000,0.000000e+00,0.0,0,0.000626,5.862207e+06
1,1,1,0,1,0,0,0,1,1,1,...,0,5,0.0,0,0.000000,0.000000e+00,42818415.0,0,0.000000,0.000000e+00
2,1,1,0,0,0,0,1,1,1,0,...,4,9,0.0,0,0.008509,1.192921e+08,0.0,0,0.000000,0.000000e+00
3,1,0,1,0,0,0,1,1,1,1,...,0,6,32694901.0,0,0.005960,9.168779e+07,0.0,0,0.000206,1.498866e+06
4,1,0,1,0,0,0,1,1,1,1,...,7,0,32694901.0,0,0.000863,1.209704e+07,0.0,0,0.000618,5.820505e+06
5,1,0,1,0,0,0,1,1,1,1,...,1,1,32694901.0,0,0.000863,1.209704e+07,38245121.0,0,0.000000,0.000000e+00
6,1,0,1,0,0,0,1,1,1,1,...,3,6,37255203.0,0,0.005673,8.857262e+07,0.0,0,0.000209,1.509604e+06
7,1,0,1,0,0,0,1,1,1,1,...,5,2,66507014.0,0,0.000288,3.910270e+06,38245121.0,0,0.000418,3.967296e+06
8,1,1,0,0,0,0,1,1,1,1,...,3,5,66507014.0,0,0.003124,4.363181e+07,0.0,0,0.000415,3.949812e+06
9,1,0,1,0,0,0,1,1,1,1,...,3,1,66507014.0,0,0.000575,8.981872e+06,0.0,0,0.000621,5.831244e+06


In [ ]:
convergence_metrics = {}
for scenario in scenario_ids:
    convergence_metrics[scenario] = []

for scenario in scenario_ids:
    hv = HypervolumeMetric(policy_dict[scenario], problem)
    gd = GenerationalDistanceMetric(policy_dict[scenario], problem, d=1)
    ei = EpsilonIndicatorMetric(policy_dict[scenario], problem)
    ig = InvertedGenerationalDistanceMetric(policy_dict[scenario], problem, d=1)
    sm = SpacingMetric(problem)

    
    for archives in archive_list:
        metrics = []
        for nfe, archive in archives.items():
            scores = {
                "generational_distance": gd.calculate(archive),
                "hypervolume": hv.calculate(archive),
                "epsilon_indicator": ei.calculate(archive),
                "inverted_gd": ig.calculate(archive),
                "spacing": sm.calculate(archive),
                "nfe": int(nfe),
            }
            metrics.append(scores)
        metrics = pd.DataFrame.from_dict(metrics)

        # sort metrics by number of function evaluations
        metrics.sort_values(by="nfe", inplace=True)
        convergence_metrics.append(metrics)
    

In [ ]:
sns.set_style("white")
fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

ax1, ax2, ax3, ax4, ax5, ax6 = axes

for metrics, convergence in zip(metrics_by_seed, convergences):
    ax1.plot(metrics.nfe, metrics.hypervolume)
    ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel("$\epsilon$ progress")

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

ax6.set_xlabel("nfe")


sns.despine(fig)

plt.show()